![alt text](img/LM.png)
# Kurs: Warsztaty Machine Learning w Pythonie
## Piotr Ćwiakowski

### Lekcja 8. Model lasów losowych

#### Spis treści

1. Bikesharing case study

In [1]:
# Wczytanie podstawowych pakietów
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Wydrukowanie wykresów
%matplotlib inline

In [2]:
# Wczytanie danych
bs = pd.read_csv("datasets/bikesharing_prepared.csv", index_col = 0)

In [3]:
# Dane - kilka pierwszych wierszy
bs.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,date,hour,weekday,month
0,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.84,14.395,81,0.0,16,2011-01-01,0,Saturday,January
1,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.02,13.635,80,0.0,40,2011-01-01,1,Saturday,January
2,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.02,13.635,80,0.0,32,2011-01-01,2,Saturday,January
3,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.84,14.395,75,0.0,13,2011-01-01,3,Saturday,January
4,Spring,0,0,Clear + Few clouds + Partly cloudy + Partly c...,9.84,14.395,75,0.0,1,2011-01-01,4,Saturday,January


In [4]:
# Stwórzmy dla wygody listę zmiennych objaśniających (features)
features = bs.columns.tolist() # zapiszmy nazwy kolumn jako zmienne
features.remove("count") # zmienna objaśniana
features.remove("date") # dla uproszczenia usuniemy datę

# Wydrukujmy listę zmiennych:
print(features)

['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday', 'month']


In [5]:
# Wczytanie z sklearna potrzebnych funkcji:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
# Zainicjowanie przykładowego modelu i zapisanie do obiektu rf
rf = RandomForestRegressor(n_estimators=100, # liczba drzew w lesie losowym
                           max_depth=7, # maksymalna głębokość
                           min_samples_split=10, # minimalna liczba obserwacji w gałęzi
                           min_samples_leaf=3, # minimalna liczba obserwacji na liściu
                           max_features = 'auto') # liczba kolumn losowanych do pojedynczego
# Dopasowanie modelu do konkretnych danych
rf.fit(X = bs[features], y = bs["count"])

ValueError: could not convert string to float: 'Spring'

Mamy błąd - wynika on z tego, że część danych jest w formie tekstowej, a funkcja szacująca model wymaga danych numerycznych. Zrekodujmy dane tekstowe, wykorzystując do tego odpowiednie narzędzia z biblioteki sklearn.

In [7]:
# import narzędzia z biblioteki sklearn
from sklearn import preprocessing

In [8]:
# Inicjujemy obiekt do kodowania, który będzie przechowywał mapę wartości
le = preprocessing.LabelEncoder()
# Dopasowujemy dane (tworzymy mapę)
le.fit([1, 2, 2, 6])
# Od tego momentu możemy mapę nakładać
le.transform([1, 1, 2, 6])

array([0, 0, 1, 2], dtype=int64)

In [9]:
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
print(le.transform(["paris", "paris", "tokyo", "amsterdam"]))
print(le.classes_)

[1 1 2 0]
['amsterdam' 'paris' 'tokyo']


In [10]:
for feature in ['season', 'weather', 'weekday', 'month']:
    # Iniciujemy obiekt do kodowania, który będzie przechowywał mapę
    le = preprocessing.LabelEncoder()
    # Dopasowujemy kolumnę (tworzymy mapę) i od razu ją nakładamy na zmienną, na której robiliśmy dopasowanie
    bs[feature] = le.fit_transform(bs[feature])

Wyświetlmy nagłówek data frame'u w celu zbadania zmian:

In [11]:
bs.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,date,hour,weekday,month
0,1,0,0,0,9.84,14.395,81,0.0,16,2011-01-01,0,2,4
1,1,0,0,0,9.02,13.635,80,0.0,40,2011-01-01,1,2,4
2,1,0,0,0,9.02,13.635,80,0.0,32,2011-01-01,2,2,4
3,1,0,0,0,9.84,14.395,75,0.0,13,2011-01-01,3,2,4
4,1,0,0,0,9.84,14.395,75,0.0,1,2011-01-01,4,2,4


Efekt zgodnie z zamierzeniem spróbujmy reestymować model:

In [12]:
# Dopasowanie modelu do danych:
rf.fit(bs[features].values, bs["count"].values)

preds = rf.predict(bs[features].values)
print("Mean squared error: %.2f" % mean_squared_error(bs["count"].values, preds))
print('R2 score: %.2f' % r2_score(bs["count"].values, preds))

Mean squared error: 6814.05
R2 score: 0.79


Błąd został zmierzony na tych samych danych, na których został oszacowany, mamy zatem do czynienia z przetrenowaniem. Wykonajmy walidację krzyżową 5 razy składaną.

In [13]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5) # stworzenie modelu do generowania foldów treningowych i testowych
mses = list() # Obiekt przechowujący wyniki z kolejnych foldów

In [14]:
# Stworzenie modelu z określonymi hiperparametrami (domyślnymi)
rf = RandomForestRegressor(n_estimators=100, # liczba drzew 
                           max_depth = None, # maksymalna głębokość drzewa 
                           min_samples_split=2, # minimalna liczba obserwacji w gałęzi do dokonania podziału
                           min_samples_leaf=1, # miniamalna liczba obserwacji na liściu 
                           max_features='auto') # liczba zmiennych wykorzystywanych w każdym drzewie

In [15]:
# Pętla po foldach
for train, test in kf.split(bs.index.values): 
    # dopasowanie modelu do zbioru treningowego
    rf.fit(bs.iloc[train][features], bs.iloc[train]["count"])
    # wygenerowanie predykcji na zbiorze treningowym i testowym
    predsTrain = rf.predict(bs.iloc[train][features])
    predsTest = rf.predict(bs.iloc[test][features])
    # Wydruk wyników
    print("Train Mean squared error: %.2f" % mean_squared_error(bs.iloc[train]["count"], predsTrain))
    print("Test Mean squared error: %.2f" % mean_squared_error(bs.iloc[test]["count"], predsTest))
    # Zapisanie błedu testowego do listy wyników
    mses.append(mean_squared_error(bs.iloc[test]["count"], predsTest))
print(np.mean(mses))

Train Mean squared error: 576.98
Test Mean squared error: 13128.68
Train Mean squared error: 514.48
Test Mean squared error: 11969.95
Train Mean squared error: 585.02
Test Mean squared error: 8747.67
Train Mean squared error: 477.57
Test Mean squared error: 14814.54
Train Mean squared error: 478.66
Test Mean squared error: 15594.31
12851.029458400575


Wykonajmy prosty tuning hiperparametrów. Zacznijmy od liczby kolumn.

In [16]:
# Pętla zewnętrzna po wartościach hiperparametru
for n_col in range(1,12,2):
    mses = [] # lista wyników
    rf = RandomForestRegressor(n_estimators = 100, # liczba drzew
                               max_depth = None, # maksymalna głębokość drzewa
                               min_samples_split = 2, # minimalna liczba obserwacji w gałęzi, żeby dokonać podziału
                               min_samples_leaf = 1, # minimalna liczba obserwacji na liściu po podziale
                               max_features = n_col) # liczba kolumn losowana do oszacowania pojedynczego drzewa 
    # Pętla wewnętrzna walidująca konkretną wartość hiperparametru
    for train, test in kf.split(bs.index.values):
        # dopasowanie modelu do zbioru treningowego
        rf.fit(bs.iloc[train][features], bs.iloc[train]["count"])
        # wygenerowanie predykcji na zbiorze treningowym i testowym
        predsTrain = rf.predict(bs.iloc[train][features])
        predsTest = rf.predict(bs.iloc[test][features])
        # Zapisanie błedu testowego do listy wyników
        mses.append(mean_squared_error(bs.iloc[test]["count"], predsTest))
    print(n_col, np.mean(mses))

1 17658.56446439008
3 14579.321075103542
5 13148.778092525274
7 12662.373544142833
9 12748.983542319858
11 12897.940329503443


7 lub 9 zmiennych wydaje się najsensowniejsze. Spróbujmy teraz sprawdzić jak na wyniki wpływa narzucenie maksymalnej głębokości drzewa. Do tej pory drzewa rozrastały się do maksymalnych rozmiarów.

In [17]:
# Pętla zewnętrzna po wartościach hiperparametru
for tree_depth in range(1, 20, 3):
    mses = [] # lista wyników
    rf = RandomForestRegressor(n_estimators = 100, # liczba drzew
                               max_depth = tree_depth, # maksymalna głębokość drzewa
                               min_samples_split = 2, # minimalna liczba obserwacji w gałęzi, żeby dokonać podziału
                               min_samples_leaf = 1, # minimalna liczba obserwacji na liściu po podziale
                               max_features = 7) # liczba kolumn losowana do oszacowania pojedynczego drzewa 
    # Pętla wewnętrzna walidująca konkretną wartość hiperparametru
    for train, test in kf.split(bs.index.values):
        # dopasowanie modelu do zbioru treningowego
        rf.fit(bs.iloc[train][features], bs.iloc[train]["count"])
        # wygenerowanie predykcji na zbiorze treningowym i testowym
        predsTrain = rf.predict(bs.iloc[train][features])
        predsTest = rf.predict(bs.iloc[test][features])
        # Zapisanie błedu testowego do listy wyników
        mses.append(mean_squared_error(bs.iloc[test]["count"], predsTest))
    print(tree_depth, np.mean(mses))

1 25376.51078987686
4 19408.43107022968
7 14457.59248314973
10 12749.701048358489
13 12749.964740266334
16 12729.549364457238
19 12659.095310294666


Głębokość powyżej 10 nie daje istotnej poprawy. Wartość 10-19 są bardzo porównywalne, wpiszmy 13 - jest lepiej niż przy 13, ale drzewa są mniej skomplikowane co ogranicza ryzyko przetrenowania. Sprawdźmy inny hiperparametr - min_samples_split.

In [18]:
# Pętla zewnętrzna po wartościach hiperparametru
for minSamp in range(2, 12, 3):
    mses = [] # lista wyników
    rf = RandomForestRegressor(n_estimators = 100, # liczba drzew
                               max_depth = 10, # maksymalna głębokość drzewa
                               min_samples_split = minSamp, # minimalna liczba obserwacji w gałęzi, żeby dokonać podziału
                               min_samples_leaf = 1, # minimalna liczba obserwacji na liściu po podziale
                               max_features = 7) # liczba kolumn losowana do oszacowania pojedynczego drzewa 
    # Pętla wewnętrzna walidująca konkretną wartość hiperparametru
    for train, test in kf.split(bs.index.values):
        # dopasowanie modelu do zbioru treningowego
        rf.fit(bs.iloc[train][features], bs.iloc[train]["count"])
        # wygenerowanie predykcji na zbiorze treningowym i testowym
        predsTrain = rf.predict(bs.iloc[train][features])
        predsTest = rf.predict(bs.iloc[test][features])
        # Zapisanie błedu testowego do listy wyników
        mses.append(mean_squared_error(bs.iloc[test]["count"], predsTest))
    print(minSamp, np.mean(mses))

2 12943.965050281377
5 12899.436895593211
8 12634.339133315723
11 12840.905404587309


Interpretacja wyników na tym etapie powinna być oczywista. Oczywiście budowa, tuning i walidacja modelu jest daleko od zakończenia, m. in.: 

* optymalnych hiperparametrów należałoby poszukać w przestrzeni wielowymiarowej z wykorzystaniem algorytmu *random search*, 
* katalog zmiennych objaśniających można by poszerzyć o dodatkowe kolumny, wykorzystując techniki *feature engineering*,
* walidacja modelu powinna być staranniejsza, dodatkowo powinniśmy zbudować zbiór testowy (i wyłączyć go z walidacji krzyżowej),
* działanie modelu powinno być sprawdzone technikami explainable AI.

Chcesz wiedzieć jak to zrobić w Pythonie? Zapisz się na kurs:
https://labmasters.pl/kursy-otwarte/python/p-3/